In [1]:
MODEL = "qwen3:1.7b"

# User Query Input

In [2]:
import requests
import json
from IPython.display import display, Markdown, clear_output

def answer_this_prompt(prompt, stream=False, model=MODEL, temperature=0, format=None):
    payload = {
        "prompt": prompt,
        "model": model,
        "temperature": temperature,
        "max_new_tokens": 50, # only when stream = False work
        "format": format
    }
    headers = {
        'Content-Type': 'application/json'
    }
    endpoint = "http://localhost:11434/api/generate"

    # Send the POST request with streaming enabled
    with requests.post(endpoint, headers=headers, json=payload, stream=True) as response:
        if response.status_code == 200:
            try:
                # Process the response incrementally
                full_response = ""
                for line in response.iter_lines(decode_unicode=True):
                    if line.strip():  # Skip empty lines
                        response_json = json.loads(line)
                        chunk = response_json.get("response", "")
                        full_response += chunk
                        
                        # Render the response as Markdown
                        if stream:
                            clear_output(wait=True)
                            display(Markdown(full_response))
                        
                return full_response
            except json.JSONDecodeError as e:
                return "Failed to parse JSON: " + str(e)
        else:
            return "Failed to retrieve response: " + str(response.status_code)
        
def multiple_answer_this_prompt(prompt, stream=False, model=MODEL, temperature=0, format=None, n_answers=1):
    answers = []
    for _ in range(n_answers):
        answer = answer_this_prompt(prompt, stream=stream, model=model, temperature=temperature, format=format)
        answers.append(answer)
    return answers

In [9]:
# Example usage
# ans = answer_this_prompt("What is the Big Bang theory?", stream=True)
# print("------------------------")
# # print(ans)

In [3]:
bn_path = "./nets/collection/"
from bni_netica.bni_netica import *
from bni_netica.bni_netica import Net

# berkeleyAdmissionNet = Net(bn_path+"Berkeley Admissions.neta")


Loading Netica


In [4]:
CancerNeapolitanNet = Net(bn_path+"Cancer Neapolitan.neta")
ChestClinicNet = Net(bn_path+"ChestClinic.neta")
ClassifierNet = Net(bn_path+"Classifier.neta")
CoronaryRiskNet = Net(bn_path+"Coronary Risk.neta")
FireNet = Net(bn_path+"Fire.neta")
MendelGeneticsNet = Net(bn_path+"Mendel Genetics.neta")
RatsNet = Net(bn_path+"Rats.neta")
WetGrassNet = Net(bn_path+"Wet Grass.neta")
RatsNoisyOr = Net(bn_path+"Rats_NoisyOr.dne")
Derm = Net(bn_path+"Derm 7.9 A.dne")

In [ ]:
BN = ""

for node in ChestClinicNet.nodes():
	# print(f"{node.name()} -> {[child.name() for child in node.children()]}")
	BN += f"{node.name()} -> {[child.name() for child in node.children()]}\n"
	
print(BN)

In [36]:
from pydantic import BaseModel

class Query(BaseModel):
  fromNode: str
  toNode: str

PROMPT = """In this Bayesian Networks: {BN}, is {fromNode} connected to {toNode}?"""

In [ ]:
fromNode = "Smoking"
toNode = "Cancer"
input = PROMPT.format(BN=BN, fromNode=fromNode, toNode=toNode) 

ans = answer_this_prompt(input, format=Query.model_json_schema())
query = Query.model_validate_json(ans)
print(query)

NameError: name 'BN' is not defined

In [ ]:
queryFromNode = query.fromNode
queryToNode = query.toNode
print(f"Is {queryFromNode} connected to {queryToNode}?")

Is Smoking connected to Cancer?


In [ ]:
def isConnected(net, fromNode, toNode):
  relatedNodes = net.node(fromNode).getRelated("d_connected")
  for node in relatedNodes:
    if node.name() == toNode:
      return True
  return False

In [ ]:
print(f"Is {queryFromNode} connected to {queryToNode}? {'Yes' if isConnected(ChestClinicNet, queryFromNode, queryToNode) else 'No'}")

Is Smoking connected to Cancer? Yes


In [ ]:
def explainConnection(net, fromNode, toNode):
    if isConnected(net, fromNode, toNode):
        return f"Yes, {fromNode} is d-connected to {toNode}, which means that entering evidence for {fromNode} would change the probability of {toNode} and vice versa."
    else:
        return f"No, {fromNode} is not d-connected to {toNode}, which means that entering evidence for {fromNode} would not change the probability of {toNode}."

In [ ]:
print(explainConnection(ChestClinicNet, queryNode1, queryNode2))

Yes, Smoking is d-connected to Cancer, which means that entering evidence for Smoking would change the probability of Cancer and vice versa.


# CHECKING CHECKING CHECKING

In [37]:
def correctIdentification(net, fromNode, toNode):
    BN = ""
    
    for node in net.nodes():
        BN += f"{node.name()} -> {[child.name() for child in node.children()]}\n"

    input = PROMPT.format(BN=BN, fromNode=fromNode, toNode=toNode)
    ans = answer_this_prompt(input, format=Query.model_json_schema())
    query = Query.model_validate_json(ans)
    queryFromNode = query.fromNode
    queryToNode = query.toNode

    return queryFromNode == fromNode and queryToNode == toNode, queryFromNode, queryToNode

In [ ]:
node1 = "Smoking"
node2 = "Cancer"
print(f"Correct identification: {correctIdentification(ChestClinicNet, node1, node2)}")

Correct identification: True


In [ ]:
import random 
def pickTwoRandomNodes(net):
    nodes = net.nodes()
    if len(nodes) < 2:
        return None, None
    node1, node2 = random.sample(nodes, 2)
    return node1.name(), node2.name()

In [15]:
print(pickTwoRandomNodes(ChestClinicNet))

('Dyspnea', 'XRay')


In [41]:
def printNet(net):
    for node in net.nodes():
        print(f"{node.name()} -> {[child.name() for child in node.children()]}")

In [ ]:
listOfNets = [CancerNeapolitanNet, ChestClinicNet, ClassifierNet, CoronaryRiskNet, FireNet, MendelGeneticsNet, RatsNet, WetGrassNet, RatsNoisyOr, Derm]
total = 0
correct = 0

for net in listOfNets:
    for _ in range(10):
      total += 1
      fromNode, toNode = pickTwoRandomNodes(net)
      if fromNode and toNode:
          correctIdentified, queryFromNode, queryToNode = correctIdentification(net, fromNode, toNode)
          if correctIdentified:
            correct += 1
          else:
            print(f"Incorrect identification for {net.name()}")
            printNet(net)
            print()
            print("Expected:", fromNode, "->", toNode)
            print("Reality:", queryFromNode, "->", queryToNode)
            print("----------------------------------------------------")

print(f"Total: {total}, Correct: {correct}, Accuracy: {correct/total:.2%}")